<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/DeutschJozsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The Function Classification Problem

**The Math:** Determine if a function is constant or balanced.
$$ f: \{0,1\}^n \to \{0,1\} $$

- **Constant:** $f(x) = 0$ for all $x$, OR $f(x) = 1$ for all $x$
- **Balanced:** $f(x) = 0$ for exactly half of inputs, $f(x) = 1$ for the other half

**Classical Complexity:**
- Best case: 2 queries (if you get different results)
- Worst case: $2^{n-1} + 1$ queries (need to check more than half)

**Quantum Complexity:** 1 query!

### The Quantum Translation: Deutsch-Jozsa Algorithm

The Deutsch-Jozsa algorithm was one of the first to demonstrate quantum advantage. It determines whether a function is constant or balanced with a single quantum query.

**The Logic:**
1. **Initialize:** Put all $n$ input qubits in superposition using Hadamard gates.
2. **Oracle Query:** Apply the quantum oracle $U_f$ once.
3. **Interference:** Apply Hadamard gates again to create interference.
4. **Measure:** If all input qubits are $|0\rangle$, function is constant; otherwise, balanced.

**Analogy for Python Devs:**
Think of this as `classify_function()` - determine if a black-box function returns the same value for all inputs or 50/50 values.

### The Qiskit Implementation

We'll implement Deutsch-Jozsa for 2 input qubits (4 possible inputs).

In [1]:
!pip install qiskit qiskit-aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.3 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# -------------------------------------------------------
# ORACLE DEFINITIONS
# -------------------------------------------------------

def create_constant_oracle(value, n_qubits):
    """
    Creates an oracle for a constant function.

    Constant 0: f(x) = 0 for all x
    Constant 1: f(x) = 1 for all x

    For constant 0: Do nothing (identity)
    For constant 1: Apply X to output qubit
    """
    oracle = QuantumCircuit(n_qubits + 1)  # +1 for output qubit

    if value == 1:
        oracle.x(n_qubits)  # Flip output qubit

    return oracle

def create_balanced_oracle(balanced_type, n_qubits):
    """
    Creates an oracle for a balanced function.

    Balanced means f(x) = 1 for exactly half of inputs.

    Examples for 2 input qubits:
    - Type 1: f(x) = x_0 (output equals first bit)
    - Type 2: f(x) = x_1 (output equals second bit)
    - Type 3: f(x) = x_0 XOR x_1 (output is XOR of bits)
    """
    oracle = QuantumCircuit(n_qubits + 1)

    if balanced_type == 1:
        # f(x) = x_0: CNOT from qubit 0 to output
        oracle.cx(0, n_qubits)
    elif balanced_type == 2:
        # f(x) = x_1: CNOT from qubit 1 to output
        oracle.cx(1, n_qubits)
    elif balanced_type == 3:
        # f(x) = x_0 XOR x_1: CNOT from both
        oracle.cx(0, n_qubits)
        oracle.cx(1, n_qubits)

    return oracle

# -------------------------------------------------------
# DEUTSCH-JOZSA CIRCUIT
# -------------------------------------------------------

def create_deutsch_jozsa_circuit(oracle, n_qubits):
    """
    Creates the complete Deutsch-Jozsa circuit.

    Structure:
    1. Initialize output qubit to |-> (X then H)
    2. Apply H to all input qubits
    3. Apply oracle
    4. Apply H to all input qubits
    5. Measure input qubits
    """
    # Total qubits: n input + 1 output
    total_qubits = n_qubits + 1

    qc = QuantumCircuit(total_qubits, n_qubits)

    # -------------------------------------------------------
    # STEP 1: Initialize output qubit to |->
    # |-> = (|0> - |1>) / sqrt(2)
    # This enables phase kickback
    # -------------------------------------------------------
    qc.x(n_qubits)  # Flip to |1>
    qc.h(n_qubits)  # Apply H to get |->

    # -------------------------------------------------------
    # STEP 2: Apply H to all input qubits
    # Creates superposition: (1/sqrt(2^n)) * sum|x>
    # -------------------------------------------------------
    for i in range(n_qubits):
        qc.h(i)

    qc.barrier()

    # -------------------------------------------------------
    # STEP 3: Apply Oracle
    # The oracle encodes f(x) into phase
    # -------------------------------------------------------
    qc.compose(oracle, inplace=True)
    qc.barrier()

    # -------------------------------------------------------
    # STEP 4: Apply H to all input qubits again
    # Creates interference pattern
    # -------------------------------------------------------
    for i in range(n_qubits):
        qc.h(i)

    qc.barrier()

    # -------------------------------------------------------
    # STEP 5: Measure input qubits
    # All zeros = constant, anything else = balanced
    # -------------------------------------------------------
    qc.measure(range(n_qubits), range(n_qubits))

    return qc

# -------------------------------------------------------
# RUN EXPERIMENTS
# -------------------------------------------------------

def run_deutsch_jozsa(oracle_type, n_qubits=2):
    """
    Runs the Deutsch-Jozsa algorithm with a given oracle type.
    """
    # Create oracle
    if oracle_type == 'constant_0':
        oracle = create_constant_oracle(0, n_qubits)
        expected = 'constant'
    elif oracle_type == 'constant_1':
        oracle = create_constant_oracle(1, n_qubits)
        expected = 'constant'
    else:
        oracle = create_balanced_oracle(
            int(oracle_type.split('_')[1]), n_qubits)
        expected = 'balanced'

    # Create circuit
    qc = create_deutsch_jozsa_circuit(oracle, n_qubits)

    # Run simulation
    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    job = simulator.run(compiled, shots=1000)
    counts = job.result().get_counts()

    # Determine result
    all_zeros = '0' * n_qubits
    if all_zeros in counts and len(counts) == 1:
        result = 'constant'
    else:
        result = 'balanced'

    return qc, counts, result, expected, oracle

# -------------------------------------------------------
# Test all oracle types
# -------------------------------------------------------

print("=== Deutsch-Jozsa Algorithm ===")
print("\nDetermining if a function is constant or balanced")
print("with a SINGLE query vs 2^(n-1)+1 classical queries!\n")

oracle_types = ['constant_0', 'constant_1', 'balanced_1', 'balanced_2', 'balanced_3']

for oracle_type in oracle_types:
    qc, counts, result, expected, oracle = run_deutsch_jozsa(oracle_type)

    print(f"\n--- Oracle: {oracle_type} ---")
    print(f"Expected: {expected}")
    print(f"Measured: {result}")
    print(f"Counts: {counts}")
    print(f"Correct: {'Yes' if result == expected else 'No'}")

print("\n" + "="*50)
print("Circuit Diagram (for balanced_1 oracle):")
print("="*50)
qc, counts, result, expected, oracle = run_deutsch_jozsa('balanced_1')
print("\nOracle:")
print(oracle.draw(output='text'))
print("\nFull Circuit:")
print(qc.draw(output='text', fold=60))

=== Deutsch-Jozsa Algorithm ===

Determining if a function is constant or balanced
with a SINGLE query vs 2^(n-1)+1 classical queries!


--- Oracle: constant_0 ---
Expected: constant
Measured: constant
Counts: {'00': 1000}
Correct: Yes

--- Oracle: constant_1 ---
Expected: constant
Measured: constant
Counts: {'00': 1000}
Correct: Yes

--- Oracle: balanced_1 ---
Expected: balanced
Measured: balanced
Counts: {'01': 1000}
Correct: Yes

--- Oracle: balanced_2 ---
Expected: balanced
Measured: balanced
Counts: {'10': 1000}
Correct: Yes

--- Oracle: balanced_3 ---
Expected: balanced
Measured: balanced
Counts: {'11': 1000}
Correct: Yes

Circuit Diagram (for balanced_1 oracle):

Oracle:
          
q_0: ──■──
       │  
q_1: ──┼──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘

Full Circuit:
     ┌───┐      ░       ░ ┌───┐ ░ ┌─┐   
q_0: ┤ H ├──────░───■───░─┤ H ├─░─┤M├───
     ├───┤      ░   │   ░ ├───┤ ░ └╥┘┌─┐
q_1: ┤ H ├──────░───┼───░─┤ H ├─░──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐ ░ └───┘ ░  ║ └╥┘
q_2: ┤ X ├┤ H ├─

### Understanding the Translation

1. **$f: \{0,1\}^n \to \{0,1\}$ (The Function):**
   - Classical: Query the function many times to determine its type.
   - Quantum: Query once using superposition and interference.

2. **Phase Kickback:**
   - The output qubit in state $|\rangle$ enables phase encoding.
   - When oracle is applied, $f(x)$ is encoded as a phase $(-1)^{f(x)}$.

3. **Interference:**
   - The second round of Hadamard gates creates interference.
   - For constant functions: All amplitudes constructively interfere at $|0...0\rangle$.
   - For balanced functions: Destructive interference at $|0...0\rangle$.

### Why is this useful?

**First Quantum Advantage:** Deutsch-Jozsa was one of the first algorithms to prove quantum computers can solve certain problems faster than classical computers.

**Educational Value:** It demonstrates key quantum concepts:
- Superposition
- Phase kickback
- Interference

**Foundation:** The techniques used here are building blocks for more complex algorithms like Grover's and Shor's.

### Complexity Comparison

| Approach | Queries Required |
|----------|-----------------|
| Classical (worst case) | $2^{n-1} + 1$ |
| Classical (best case) | 2 |
| **Quantum (Deutsch-Jozsa)** | **1** |

### Summary for Python Developers

```python
# Classical approach
def classify_function_classical(f, n):
    results = [f(x) for x in range(2**n)]
    if all(r == 0 for r in results) or all(r == 1 for r in results):
        return 'constant'
    return 'balanced'
# Requires 2^(n-1) + 1 queries in worst case!

# Quantum approach (Deutsch-Jozsa)
def classify_function_quantum(oracle):
    # Single query!
    result = run_deutsch_jozsa(oracle)
    return result  # 'constant' or 'balanced'
```